**EJERCICIOS** 

En este ejercicio tendréis que usar la misma API que hemos usado hasta ahora y elegir otro punto de información (hasta ahora hemos usado `civil` y `astro`). Descarga los datos, limpialos y juntalos con los datos de los otros puntos de información que hemos obtenido durante esta lección. 

📌 **NOTA** Al juntar toda la información puede que haya algunas columnas que tengan la misma información. Debemos evitar información duplicada en nuestro *dataframe*, por lo que tendremos que hacer una buena selección de las columnas que queremos conservar.

💪🏽 **BONUS** Puede que algunas de las columnas que tengamos de la nueva llamada a la API tengan una pinta como esta 👇🏽

```python
[{'layer': '950mb', 'rh': 9},
 {'layer': '900mb', 'rh': 11},
 {'layer': '850mb', 'rh': 12},
 {'layer': '800mb', 'rh': 6},
 {'layer': '750mb', 'rh': 4},
 {'layer': '700mb', 'rh': 5},
 {'layer': '650mb', 'rh': 7},
 {'layer': '600mb', 'rh': 15},
 {'layer': '550mb', 'rh': 16},
 {'layer': '500mb', 'rh': 13},
 {'layer': '450mb', 'rh': 9},
 {'layer': '400mb', 'rh': 11},
 {'layer': '350mb', 'rh': 10},
 {'layer': '300mb', 'rh': 10},
 {'layer': '250mb', 'rh': 8},
 {'layer': '200mb', 'rh': -3}]
```

El objetivo de este bonus es separar cada elemento de esa lista de diccionarios en columnas diferentes. 


In [1]:
import requests 
import pandas as pd
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [5]:

df_leccion = pd.read_pickle('/home/alejandro/Desktop/datos_actualizados.pkl')
df_leccion.head(1)

,Unnamed: 0,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,0,3,9,-1,rain,1,21,71,lightrainnight,SW,2,2022-09-13,40.4165,-3.70256,Madrid,3,3


In [6]:
latitud = 40.4165  
longitud = -3.70256
producto = 'meteo'

In [7]:
def llamada_API(lat, lon, producto):
    
    # hacemos la llamada  a la API
    url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'

    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason
    if codigo_estado == 200:
        print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 402:
        print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 404:
        print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    else:
        print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

    # convertimos los resultados en un dataframe: 
    df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
    return df

In [8]:
df_hoy_meteo = llamada_API(latitud,longitud,producto)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [13]:
df_leccion.head()

,Unnamed: 0,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,0,3,9,-1,rain,1,21,71,lightrainnight,SW,2,2022-09-13,40.4165,-3.70256,Madrid,3,3
1,1,6,9,-1,rain,1,22,61,lightrainday,W,2,2022-09-13,40.4165,-3.70256,Madrid,4,2
2,2,9,9,-1,rain,1,22,53,lightrainday,S,2,2022-09-13,40.4165,-3.70256,Madrid,1,3
3,3,12,9,-1,rain,1,24,53,lightrainday,S,3,2022-09-13,40.4165,-3.70256,Madrid,1,3
4,4,15,9,2,rain,3,20,71,lightrainday,W,3,2022-09-13,40.4165,-3.70256,Madrid,2,3


In [9]:
df_hoy_meteo.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 235, 'speed':...",17,-1,13,1008,rain,2,0,285,2


In [10]:
df_leccion.columns

Index(['Unnamed: 0', 'timepoint', 'cloudcover', 'lifted_index', 'prec_type',
       'prec_amount', 'temp2m', 'rh2m', 'weather', 'wind10m.direction',
       'wind10m.speed', 'fecha', 'latitud', 'longitud', 'ciudad', 'seeing',
       'transparency'],
      dtype='object')

In [11]:
df_hoy_meteo.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed'],
      dtype='object')

In [14]:
df_hoy_meteo_limpio = df_hoy_meteo[['timepoint','highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'lifted_index', 'msl_pressure', 'snow_depth']]

In [19]:
hoy = datetime.now()
hoy = datetime.strftime(hoy,'%Y-%m-%d')

df_hoy_meteo_limpio['fecha'] =hoy
df_hoy_meteo_limpio['ciudad']= 'Madrid'
df_hoy_meteo_limpio.head()

,timepoint,highcloud,midcloud,lowcloud,rh_profile,lifted_index,msl_pressure,snow_depth,fecha,ciudad
0,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...",-1,1008,0,2022-09-14,Madrid
1,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...",-1,1008,0,2022-09-14,Madrid
2,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...",-1,1009,0,2022-09-14,Madrid
3,12,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...",-1,1009,0,2022-09-14,Madrid
4,15,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...",2,1007,0,2022-09-14,Madrid


In [20]:
df_hoy_meteo_limpio.set_index(['timepoint','ciudad'], inplace= True)
df_hoy_meteo_limpio.head()

,,highcloud,midcloud,lowcloud,rh_profile,lifted_index,msl_pressure,snow_depth,fecha
timepoint,ciudad,,,,,,,,
3,Madrid,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...",-1,1008,0,2022-09-14
6,Madrid,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...",-1,1008,0,2022-09-14
9,Madrid,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...",-1,1009,0,2022-09-14
12,Madrid,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...",-1,1009,0,2022-09-14
15,Madrid,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...",2,1007,0,2022-09-14


In [21]:
df= df_leccion.join(df_hoy_meteo_limpio, on = ['timepoint','ciudad'], lsuffix=' ', how='inner')


In [22]:
df.head()

,Unnamed: 0,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,...,seeing,transparency,highcloud,midcloud,lowcloud,rh_profile,lifted_index,msl_pressure,snow_depth,fecha
0,0,3,9,-1,rain,1,21,71,lightrainnight,SW,...,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...",-1,1008,0,2022-09-14
1,1,6,9,-1,rain,1,22,61,lightrainday,W,...,4,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...",-1,1008,0,2022-09-14
2,2,9,9,-1,rain,1,22,53,lightrainday,S,...,1,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...",-1,1009,0,2022-09-14
3,3,12,9,-1,rain,1,24,53,lightrainday,S,...,1,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...",-1,1009,0,2022-09-14
4,4,15,9,2,rain,3,20,71,lightrainday,W,...,2,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...",2,1007,0,2022-09-14


In [26]:
df_hoy_meteo_limpio.shape

(64, 8)

In [25]:
df_leccion.shape

(24, 17)

In [23]:
df.shape

(24, 25)

In [32]:
df['rh_profile'].iloc[0]

[{'layer': '950mb', 'rh': 12},
 {'layer': '900mb', 'rh': 12},
 {'layer': '850mb', 'rh': 9},
 {'layer': '800mb', 'rh': 6},
 {'layer': '750mb', 'rh': 7},
 {'layer': '700mb', 'rh': 10},
 {'layer': '650mb', 'rh': 7},
 {'layer': '600mb', 'rh': 5},
 {'layer': '550mb', 'rh': 8},
 {'layer': '500mb', 'rh': 13},
 {'layer': '450mb', 'rh': 14},
 {'layer': '400mb', 'rh': 15},
 {'layer': '350mb', 'rh': 15},
 {'layer': '300mb', 'rh': 15},
 {'layer': '250mb', 'rh': 16},
 {'layer': '200mb', 'rh': 15}]

In [ ]:
'rh_950mb' = 12
'rh_900mb ' = 12

In [33]:
dicto= {'layer': '950mb', 'rh': 12}

In [35]:
for key, value in dicto.items():
    print(key)
    print(value)
    print(value[0] +key[1])

layer
950mb
9a
rh
12


TypeError: 'int' object is not subscriptable

In [37]:
df.head()

,Unnamed: 0,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,...,seeing,transparency,highcloud,midcloud,lowcloud,rh_profile,lifted_index,msl_pressure,snow_depth,fecha
0,0,3,9,-1,rain,1,21,71,lightrainnight,SW,...,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...",-1,1008,0,2022-09-14
1,1,6,9,-1,rain,1,22,61,lightrainday,W,...,4,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...",-1,1008,0,2022-09-14
2,2,9,9,-1,rain,1,22,53,lightrainday,S,...,1,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...",-1,1009,0,2022-09-14
3,3,12,9,-1,rain,1,24,53,lightrainday,S,...,1,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...",-1,1009,0,2022-09-14
4,4,15,9,2,rain,3,20,71,lightrainday,W,...,2,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...",2,1007,0,2022-09-14


In [46]:
lista = [['a','x'],['b','z'],['c','d']]
pd.DataFrame(pd.Series(lista))

,0
0,"[a, x]"
1,"[b, z]"
2,"[c, d]"


In [47]:
lista = [['a','x'],['b'],['c','d']]
pd.DataFrame(pd.Series(lista))[0].apply(pd.Series)

,0,1
0,a,x
1,b,NaN
2,c,d


In [42]:
lista = ['a','b','c']
pd.DataFrame(pd.Series(lista))

,0
0,a
1,b
2,c


In [52]:
x = df['rh_profile'].apply(pd.Series)
x[0].apply(pd.Series)['layer'][0]

'950mb'

In [53]:
x = df['rh_profile'].apply(pd.Series)
x[0].apply(pd.Series)['rh'][0]

12

In [55]:
x = df['rh_profile'].apply(pd.Series)

for values in range(len(x.columns)):
    nombre='rh' + str(x[values].apply(pd.Series)['layer'][0])
    valores = list(x[values].apply(pd.Series)['rh'])
    df.insert(values, nombre, valores)

In [57]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

In [58]:
df.head()

,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,Unnamed: 0,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency,highcloud,midcloud,lowcloud,rh_profile,lifted_index,msl_pressure,snow_depth,fecha
0,12,12,9,6,7,10,7,5,8,13,14,15,15,15,16,15,0,3,9,-1,rain,1,21,71,lightrainnight,SW,2,2022-09-13,40.4165,-3.70256,Madrid,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...",-1,1008,0,2022-09-14
1,8,9,8,6,3,8,0,-1,3,2,0,13,16,16,16,16,1,6,9,-1,rain,1,22,61,lightrainday,W,2,2022-09-13,40.4165,-3.70256,Madrid,4,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...",-1,1008,0,2022-09-14
2,6,6,6,2,6,0,-2,-2,2,3,1,13,16,16,16,1,2,9,9,-1,rain,1,22,53,lightrainday,S,2,2022-09-13,40.4165,-3.70256,Madrid,1,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...",-1,1009,0,2022-09-14
3,4,3,5,7,5,1,-2,0,7,7,-1,-2,8,16,16,-1,3,12,9,-1,rain,1,24,53,lightrainday,S,3,2022-09-13,40.4165,-3.70256,Madrid,1,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...",-1,1009,0,2022-09-14
4,2,1,3,4,7,11,-2,-2,-1,1,5,4,5,6,5,0,4,15,9,2,rain,3,20,71,lightrainday,W,3,2022-09-13,40.4165,-3.70256,Madrid,2,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...",2,1007,0,2022-09-14
